In [5]:
import numpy as np
import json
import re
import tensorflow as tf
import warnings

import spacy
nlp = spacy.load('en_core_web_sm')

warnings.filterwarnings('ignore')

In [2]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 11.7/11.7 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 2.5 MB/s eta 0:00:00
  Using cached preshed-3.0.6-cp310-cp310-win_amd64.whl (112 kB)
     -------------------------------------- 448.3/448.3 KB 2.0 MB/s eta 0:00:00
     -------------------------------------- 126.0/126.0 KB 1.5 MB/s eta 0:00:00
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
     ---------------------------------------- 1.0/1.0 MB 2.2 MB/s eta 0:00:00
  Using cached cymem-2.0.6-cp310-cp310-win_amd64.whl (36 kB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
''' reading data '''
with open('Intent.json', 'rb') as file:
    data = json.load(file)

In [7]:
''' preprocessing '''
def pre_processing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [9]:
'''get text and intent title from json data'''
inputs, targets = [], []
cls = []
intent_doc = {}

for i in data['intents']:
    if i['intent'] not in cls:
        cls.append(i['intent'])
        
    if i['intent'] not in intent_doc:
        intent_doc[i['intent']] = []
        
    for text in i['text']:
        inputs.append(pre_processing(text))
        targets.append(i['intent'])
        
    for response in i['responses']:
        intent_doc[i['intent']].append(response)

In [10]:
''' tokenize data '''
def token_data(inp_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(inp_list)
    
    inp_seq = tokenizer.texts_to_sequences(inp_list)

    ''' adding padding '''
    inp_seq = tf.keras.preprocessing.sequence.pad_sequences(inp_seq, padding='pre')
    
    return tokenizer, inp_seq

'''preprocess input data'''
tokenizer, inp_tensor = token_data(inputs)

In [11]:
def cr_cat_target(targets):
    word = {}
    cat_t = []
    counter=0
    
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        cat_t.append(word[trg])
    
    cat_tensor = tf.keras.utils.to_categorical(cat_t, num_classes=len(word), dtype='int32')
    return cat_tensor, dict((v,k) for k, v in word.items())

'''preprocess output data'''
target_tensor, target_idx_word = cr_cat_target(targets)

In [12]:
print('input shape: {} and output shape: {}'.format(inp_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


In [13]:
''' Build Model '''
''' hyperparameters'''
epochs=50
vocab_size = len(tokenizer.word_index) + 1
embed_dim = 512
units=128
target_len = target_tensor.shape[1]

''' Model '''
model = tf.keras.models.Sequential([
    ## Embedding Layer 
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    ## Hidden Layer 
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    ## Classification Layer 
    tf.keras.layers.Dense(target_len, activation='softmax')])

In [14]:
''' Compile the model '''
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-2), loss='categorical_crossentropy', metrics=['accuracy'])

''' lets see how model looks like '''
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
__________________________________________________

In [10]:
''' EarlyStopping'''
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

''' training '''
model.fit(inp_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 0s 8ms/step - loss: 3.1203 - accuracy: 0.0699
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 2.6396 - accuracy: 0.2238
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 1.9043 - accuracy: 0.4545
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 1.2231 - accuracy: 0.5664
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 0.8546 - accuracy: 0.7203
Epoch 6/50
5/5 [==============================] - 0s 7ms/step - loss: 0.3908 - accuracy: 0.8881
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 0.2393 - accuracy: 0.9301
Epoch 8/50
5/5 [==============================] - 0s 7ms/step - loss: 0.1778 - accuracy: 0.9301
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 0.2322 - accuracy: 0.9441
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0692 - accuracy: 0.9720
Epoch 11/50
5/5 [======================

In [ ]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.
